# Match Stats 

In [2]:
from fotmob_api import FotmobAPI
import pandas as pd

# Initialize client
client = FotmobAPI()

# Get fixtures for a specific league
# get league id
all_leagues = client.get_league_all()
df_country_leagues = pd.json_normalize(all_leagues["countries"],
                                       record_path=["leagues"],
                                       meta=["ccode", "name", "localizedName"],
                                       record_prefix="league")
league_id = df_country_leagues[(df_country_leagues["leaguename"]=="Premier League") &
                               (df_country_leagues["name"]=="England")]["leagueid"].item()

# get fixtures
fixtures = client.get_fixtures(id=league_id, season="2023/2024")
df_fixtures = pd.json_normalize(fixtures, max_level=1, sep="_")
df_fixtures.head()

,id,pageUrl,displayTournament,notStarted,opponent_id,opponent_name,opponent_score,home_id,home_name,home_score,...,away_name,away_score,status_utcTime,status_finished,status_started,status_cancelled,status_scoreStr,status_reason,status_ongoing,status_liveTime
0,4193450,/matches/burnley-vs-manchester-city/2ai7j8#419...,True,False,8191,Burnley,0.0,8191,Burnley,0.0,...,Man City,3.0,2023-08-11T19:00:00.000Z,True,True,False,0 - 3,"{'short': 'FT', 'shortKey': 'fulltime_short', ...",NaN,NaN
1,4193451,/matches/arsenal-vs-nottingham-forest/3bf4p3#4...,True,False,9825,Arsenal,2.0,9825,Arsenal,2.0,...,Nottm Forest,1.0,2023-08-12T12:00:00.000Z,True,True,False,2 - 1,"{'short': 'FT', 'shortKey': 'fulltime_short', ...",NaN,NaN
2,4193452,/matches/west-ham-united-vs-afc-bournemouth/2h...,True,False,8678,Bournemouth,1.0,8678,Bournemouth,1.0,...,West Ham,1.0,2023-08-12T14:00:00.000Z,True,True,False,1 - 1,"{'short': 'FT', 'shortKey': 'fulltime_short', ...",NaN,NaN
3,4193453,/matches/luton-town-vs-brighton-hove-albion/2u...,True,False,10204,Brighton,4.0,10204,Brighton,4.0,...,Luton,1.0,2023-08-12T14:00:00.000Z,True,True,False,4 - 1,"{'short': 'FT', 'shortKey': 'fulltime_short', ...",NaN,NaN
4,4193454,/matches/everton-vs-fulham/2uevfp#4193454,True,False,8668,Everton,0.0,8668,Everton,0.0,...,Fulham,1.0,2023-08-12T14:00:00.000Z,True,True,False,0 - 1,"{'short': 'FT', 'shortKey': 'fulltime_short', ...",NaN,NaN


In [3]:
# Create match description column
df_fixtures["description"] = (df_fixtures["home_name"] + " - " +
                              df_fixtures["away_name"])
df_fixtures["date"] = pd.to_datetime(df_fixtures["status_utcTime"]).dt.strftime('%Y-%m-%d')

# Get match id for Chelsea - Man City on 12th of November
match_id = df_fixtures[(df_fixtures["description"] == "Chelsea - Man City") &
                       (df_fixtures["date"] == "2023-11-12")]["id"].item()

# Get match details
match_details = client.get_match_details(match_id=match_id)

# overall match stats
df_match_stats = pd.json_normalize(match_details["content"]["stats"]["Periods"]["All"]["stats"],
                                   record_path=["stats"], meta=["title"], meta_prefix="meta_")
df_match_stats[["home_stats", "away_stats"]] = pd.DataFrame(df_match_stats["stats"].tolist(), index=df_match_stats.index)
# shots
df_match_shots = pd.DataFrame(match_details["content"]["shotmap"]["shots"])

In [4]:
# show overall match stats
df_match_stats.head(9)

,title,key,stats,type,highlighted,meta_title,home_stats,away_stats
0,Ball possession,BallPossesion,"[45, 55]",graph,away,Top stats,45,55
1,Expected goals (xG),expected_goals,"[2.95, 2.88]",text,home,Top stats,2.95,2.88
2,Total shots,total_shots,"[17, 15]",text,home,Top stats,17,15
3,Shots on target,ShotsOnTarget,"[9, 10]",text,away,Top stats,9,10
4,Big chances,big_chance,"[5, 5]",text,equal,Top stats,5,5
5,Big chances missed,big_chance_missed_title,"[2, 2]",text,equal,Top stats,2,2
6,Accurate passes,accurate_passes,"[377 (85%), 477 (90%)]",text,away,Top stats,377 (85%),477 (90%)
7,Fouls committed,fouls,"[13, 15]",text,home,Top stats,13,15
8,Corners,corners,"[3, 3]",text,equal,Top stats,3,3


In [5]:
# show shots
df_match_shots[["eventType", "playerName", "x", "y", "min", "shotType", "situation", "expectedGoals", "expectedGoalsOnTarget"]].head()

,eventType,playerName,x,y,min,shotType,situation,expectedGoals,expectedGoalsOnTarget
0,AttemptSaved,Reece James,91.400000,19.234904,1,RightFoot,RegularPlay,0.030917,NaN
1,AttemptSaved,Cole Palmer,93.100000,21.056561,12,LeftFoot,RegularPlay,0.038994,NaN
2,AttemptSaved,Julián Álvarez,99.114035,19.935541,16,RightFoot,RegularPlay,0.036596,NaN
3,AttemptSaved,Josko Gvardiol,92.200000,39.460952,17,Header,FromCorner,0.009272,0.0330
4,AttemptSaved,Conor Gallagher,82.710194,30.220000,19,RightFoot,RegularPlay,0.049839,0.0298
